In [8]:
import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [9]:
import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv()

# Initialize the Slack client with your Bot User OAuth Token
client = WebClient(token=os.getenv("SLACK_BOT_USER_OAUTH_TOKEN"))

# Specify the channel ID you want to fetch history from
channel_id = 'C057EJE6QDB'

# Function to fetch and save channel history
def fetch_channel_history(channel_id):
    try:
        # Initialize variables
        has_more = True
        cursor = None
        all_messages = []

        # Fetch messages in a loop to handle pagination
        while has_more:
            response = client.conversations_history(
                channel=channel_id,
                cursor=cursor,
                limit=200  # Maximum is 200
            )
            messages = response['messages']
            all_messages.extend(messages)
            has_more = response['has_more']
            cursor = response['response_metadata']['next_cursor'] if has_more else None


        print(f"Downloaded {len(all_messages)} messages from channel {channel_id}.")

        return all_messages

    except SlackApiError as e:
        print(f"Error fetching conversations: {e.response['error']}")

# Call the function
messages = fetch_channel_history(channel_id)

Downloaded 832 messages from channel C057EJE6QDB.


In [15]:
# Function to fetch user info
def fetch_user_info(user_id):
    try:
        response = client.users_info(user=user_id)
        if response['ok']:
            user_info = response['user']
            username = user_info['name']  # Username
            real_name = user_info['real_name'] if 'real_name' in user_info else None  # Full name
            return username, real_name
        else:
            print(f"Error fetching user info: {response['error']}")
            return None, None
    except SlackApiError as e:
        print(f"Error fetching user info: {e.response['error']}")
        return None, None

In [16]:
# Dictionary to cache user info
user_cache = {}

# Process messages
for message in messages:
    user_id = message.get('user')
    if user_id:
        if user_id not in user_cache:
            username, real_name = fetch_user_info(user_id)
            user_cache[user_id] = {'username': username, 'real_name': real_name}
        else:
            username = user_cache[user_id]['username']
            real_name = user_cache[user_id]['real_name']

        # Add username and real_name to the message
        message['username'] = username
        message['real_name'] = real_name

# Save messages with usernames to a JSON file
with open(f'{channel_id}_history_with_usernames.json', 'w') as f:
    json.dump(messages, f, indent=4)

print(f"Processed {len(messages)} messages from channel {channel_id}.")

Processed 832 messages from channel C057EJE6QDB.


In [ ]:
import requests

def download_file(url, filename):
    headers = {'Authorization': f'Bearer YOUR_SLACK_API_TOKEN'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded file: {filename}")
    else:
        print(f"Failed to download file: {filename}")

def fetch_channel_history_with_files(channel_id):
    try:
        has_more = True
        cursor = None
        all_messages = []

        while has_more:
            response = client.conversations_history(
                channel=channel_id,
                cursor=cursor,
                limit=200
            )
            messages = response['messages']
            all_messages.extend(messages)
            has_more = response['has_more']
            cursor = response['response_metadata']['next_cursor'] if has_more else None

            # Check for files in messages
            for message in messages:
                if 'files' in message:
                    for file_info in message['files']:
                        file_url = file_info['url_private']
                        file_name = file_info['name']
                        download_file(file_url, file_name)

        with open(f'{channel_id}_history_with_files.json', 'w') as f:
            json.dump(all_messages, f, indent=4)

        print(f"Downloaded {len(all_messages)} messages and associated files from channel {channel_id}.")

    except SlackApiError as e:
        print(f"Error fetching conversations: {e.response['error']}")

# Call the function
fetch_channel_history_with_files(channel_id)